## Mapper Agent

Action: Analyze the initial user input (question, idea, problem). Identify key entities, claims, and underlying assumptions.

Output structure: **Draft Knowledge Graph** -> A preliminary visual map showing the relationships between concepts in the *original prompt only*. (e.g. Idea A -> Assumes B)

Key Decision: The Mapper Agent reviews the prompt and determines if the requires deep research or not. If the prompt indicates that the Agent should simply try and understand the input and build a graph, it skips the deep research, otherwise, if the query requires additional knowledge then the Mapper Agent sends the prompt to a Research Agent.

In [1]:
from openai import AsyncOpenAI
from agents import Agent, trace, Runner, function_tool, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field, conlist
import asyncio
from typing import Dict
import requests
import os
from datetime import datetime
from IPython.display import Markdown, display
from collections import namedtuple

### Formatted markdown functions

In [ ]:
# Display Mapper Agent output function

def display_mapper_output_as_markdown_escaped(mapper_output_object):
    """
    Takes a MapperAgentOutput object and outputs it as a structured Markdown table, 
    escaping special characters in the Predicate column for reliable rendering.
    """
    
    # 1. Safely extract the attributes
    try:
        knowledge_triplets_list = mapper_output_object.draft_knowledge_graph
        research_status = mapper_output_object.research_needed
    except AttributeError:
        return "⚠️ ERROR: Input object missing necessary attributes."

    # 2. Build the main table
    markdown_output = "### 🧠 Draft Knowledge Graph\n\n"
    
    if not knowledge_triplets_list:
        markdown_output += "> The knowledge graph is currently empty.\n\n"
    else:
        markdown_output += "| **Subject** | **Predicate** | **Object** |\n"
        markdown_output += "| :--- | :---: | :--- |\n"
        
        for triplet in knowledge_triplets_list:
            try:
                subject = str(getattr(triplet, 'subject', 'N/A'))
                predicate = str(getattr(triplet, 'predicate', 'N/A'))
                object_ = str(getattr(triplet, 'object', 'N/A'))
            except Exception:
                subject, predicate, object_ = "ERROR", "ERROR", "ERROR"

            # --- CRITICAL FIX: Escape special Markdown/HTML characters ---
            # 1. Escape angle brackets in the predicate
            predicate = predicate.replace('<<', r'\<\<').replace('>>', r'\>\>')

            # 2. Escape pipe characters in all fields
            subject = subject.replace('|', '/')
            predicate = predicate.replace('|', '/')
            object_ = object_.replace('|', '/')
            
            # The Predicate is now displayed without bolding for max visibility
            markdown_output += f"| {subject} | {predicate} | {object_} |\n" # Note: No ** around predicate

    # 3. Add the research status footer
    status_text = "✅ No further research is needed."
    if research_status:
        status_text = "🔎 **Further research is needed.**"

    markdown_output += f"\n---\n\n**Research Status:** {status_text}"
    
    # display(Markdown(markdown_output)) 
    return markdown_output

In [ ]:
# Display Search Agent output function

def display_as_markdown_table(data_list):
    # 1. Start the Markdown table with headers
    markdown_output = "| Reason | Query |\n"
    markdown_output += "| :--- | :--- |\n"  # Separator line for alignment

    # 2. Add each row
    for item in data_list:
        # **CORRECTION**: Access elements by index [0] and [1] for a tuple
        try:
            reason = item[0] 
            query = item[1]
        except (IndexError, TypeError):
            # Fallback if the item isn't a tuple or doesn't have enough elements
            reason = "ERROR: Bad Format"
            query = "ERROR: Bad Format"

        # Escape pipe characters in the text to avoid breaking the table structure
        reason = str(reason).replace('|', '/')
        query = str(query).replace('|', '/')
        
        markdown_output += f"| {reason} | **{query}** |\n"

    # 3. Use the IPython display function to render the Markdown
    display(Markdown(markdown_output))

### Get the model

In [2]:
# Models

# get the API key
google_api_key = os.getenv("GOOGLE_API_KEY")

# connect to endpoints (necessary for Google but not OpenAI)
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

### Define Mapper Agent

#### Structured Outputs

In [3]:
class KnowledgeTriplet(BaseModel):
    """Represents a single relationship (edge) between two concepts (nodes) 
    extracted directly from the user's input."""
    
    subject: str = Field(description="The primary concept or entity (Node) in the relationship.")
    
    # Standardized predicates based on input analysis
    # Use one of the following: 'IMPLIES', 'ASSUMES', 'INQUIRES', 'FOCUSES_ON'.
    predicate: str = Field(description="The nature of the relationship (Edge) between the subject and object (e.g., 'IMPLIES', 'ASSUMES', 'INQUIRES').")
    
    object: str = Field(description="The secondary concept, claim, or assumption (Node) related to the subject.")
    
class MapperAgentOutput(BaseModel):
    """The complete structured output for the Mapper Agent."""
    
    # The list of relationships forms the Draft Knowledge Graph
    draft_knowledge_graph: list[KnowledgeTriplet] = Field(
        description="A list of KnowledgeTriplet objects representing the conceptual map of the user's input only."
    )
    
    # The key decision flag for the next stage
    research_needed: bool = Field(
        description="True if the input is a knowledge-based query ('What are the implications of X?') requiring external evidence. \
            False if it is a simple planning query ('How to build Y?')."
    )

#### Instructions

In [4]:
MAPPER_AGENT_INSTRUCTIONS = """You are the Conceptual Pathfinder Agent's Mapper component.
Your task is to analyze a single, potentially vague user input (question, idea, or problem) and convert it into a structured, conceptual map.

## MANDATORY STEPS:

1. Deconstruct the Input: Identify all key concepts, explicit claims, and implicit assumptions present in the original prompt.
2. Generate the Draft Knowledge Graph: Create a list of structured triplets that map the relationships between these identified
concepts, claims, and assumptions. **Do NOT introduce external information.**
    -- Nodes: Use the identified concepts, claims, and assumptions as your nodes (Subject/ Object).
    -- Edges (Predicates): Use standardized relationship types:
        << IMPLIES >>: A concept suggests or requires another.
        << ASSUMES >>: A concept or claim relies on an implicit or explicit assumption.
        << INQUIRIES >>: Denotes a specific question or uncertainty (e.g., a claim being questioned).
3. Determine Research Need: Decide if the core of the query is **conceptual/ knowledge-based** (e.g., What are the ethical implication of X")
requiring external research, or **procedural/ planning-based** (e.g., "How to build a shed") which can proceed directly to action planning.
Set the 'research_needed' flag accordingly.
"""

#### Agent Configuration

In [7]:
mapper_agent = Agent(
    name="Mapper Agent",
    instructions=MAPPER_AGENT_INSTRUCTIONS,
    model=gemini_model,
    output_type=MapperAgentOutput,
)

In [10]:
async def triage_and_structure_query(query: str) -> MapperAgentOutput:
    """Identify the Nodes and Edges of the input for the knowledge graph.
    Flag if research_needed is True or False to decide if external research is needed."""
    
    result = await Runner.run(mapper_agent, query)
    return result.final_output

#### Testing

In [18]:
# Testing prompts: external research expected

external_research_query_1 = "Is a four-day work week economically viable for all businesses, or does it only succeed in the tech sector?"
external_research_query_2 = "How has the rise of large language models changed the legal concept of intellectual property ownership, and what will the courts likely rule in the next five years?"
external_research_query_3 = "If we terraform Mars, what is the single greatest risk to the stability of the new atmosphere, and how much would a pilot project cost?"

In [19]:
# Testing prompts: no external research expected

no_external_research_query_1 = "I want to start a side hustle selling custom stickers. What are the first three steps I should take, and how do I calculate my initial material costs?"
no_external_research_query_2 = "Design a daily schedule for a student studying for three different college final exams who needs eight hours of sleep."
no_external_research_query_3 = "List the components required to build a basic home hydroponic system and arrange them by acquisition priority."

In [20]:
# Test: External research expected
external_result_1 = await triage_and_structure_query(external_research_query_1)
external_result_2 = await triage_and_structure_query(external_research_query_2)
external_result_3 = await triage_and_structure_query(external_research_query_3)

In [21]:
# Test: External research NOT expected
no_external_result_1 = await triage_and_structure_query(no_external_research_query_1)
no_external_result_2 = await triage_and_structure_query(no_external_research_query_2)
no_external_result_3 = await triage_and_structure_query(no_external_research_query_3)

In [22]:
# Display results: external research expected
external_results_list = [external_result_1, external_result_2, external_result_3]

for result in external_results_list:
    print("\n")
    display(Markdown(display_mapper_output_as_markdown_escaped(result)))
    

### 🧠 Draft Knowledge Graph

| **Subject** | **Predicate** | **Object** |
| :--- | :---: | :--- |
| Four-day work week | \<\<INQUIRES\>\> | Economic viability for all businesses |
| Four-day work week | \<\<INQUIRES\>\> | Success limited to tech sector |
| Economic viability for all businesses | \<\<IMPLIES\>\> | Increased productivity or reduced costs |
| Success limited to tech sector | \<\<ASSUMES\>\> | Tech sector has unique economic conditions |
| All businesses | \<\<ASSUMES\>\> | Businesses have similar economic structures |
| Four-day work week | \<\<ASSUMES\>\> | Four-day work week definition is consistent |

---

**Research Status:** 🔎 **Further research is needed.**

### 🧠 Draft Knowledge Graph

| **Subject** | **Predicate** | **Object** |
| :--- | :---: | :--- |
| Rise of Large Language Models | \<\<IMPLIES\>\> | Change in legal concept of intellectual property ownership |
| Legal concept of intellectual property ownership | \<\<INQUIRES\>\> | How has it changed? |
| Courts | \<\<INQUIRES\>\> | What will they likely rule in the next five years regarding intellectual property ownership and LLMs? |
| Courts ruling on IP ownership and LLMs | \<\<ASSUMES\>\> | Courts will rule on cases related to intellectual property ownership and LLMs in the next five years. |

---

**Research Status:** 🔎 **Further research is needed.**

### 🧠 Draft Knowledge Graph

| **Subject** | **Predicate** | **Object** |
| :--- | :---: | :--- |
| Terraforming Mars | \<\<IMPLIES\>\> | Creating a new Martian atmosphere |
| Stability of the new Martian atmosphere | \<\<INQUIRIES\>\> | What is the single greatest risk? |
| Terraforming Mars | \<\<INQUIRIES\>\> | How much would a pilot project cost? |
| Pilot project | \<\<IMPLIES\>\> | Testing terraforming methods on a small scale |
| Cost of a pilot project | \<\<ASSUMES\>\> | Feasibility of such a project |

---

**Research Status:** 🔎 **Further research is needed.**

In [23]:
# Display results: no external research expected
external_results_list = [no_external_result_1, no_external_result_2, no_external_result_3]

for result in external_results_list:
    print("\n")
    display(Markdown(display_mapper_output_as_markdown_escaped(result)))

### 🧠 Draft Knowledge Graph

| **Subject** | **Predicate** | **Object** |
| :--- | :---: | :--- |
| Starting a side hustle selling custom stickers | \<\<IMPLIES\>\> | Defining initial steps |
| Starting a side hustle selling custom stickers | \<\<IMPLIES\>\> | Calculating initial material costs |
| Defining initial steps | \<\<INQUIRIES\>\> | What are the first three steps? |
| Calculating initial material costs | \<\<INQUIRIES\>\> | How to calculate costs? |
| Starting a side hustle selling custom stickers | \<\<ASSUMES\>\> | Resources (time, money, tools) are available |

---

**Research Status:** ✅ No further research is needed.

### 🧠 Draft Knowledge Graph

| **Subject** | **Predicate** | **Object** |
| :--- | :---: | :--- |
| Daily schedule | \<\<IMPLIES\>\> | Time management |
| Daily schedule | \<\<IMPLIES\>\> | Study blocks |
| Daily schedule | \<\<IMPLIES\>\> | Breaks |
| Student | \<\<IMPLIES\>\> | College final exams |
| Student | \<\<IMPLIES\>\> | Need for focused study |
| College final exams | \<\<IMPLIES\>\> | Three different subjects |
| Three different subjects | \<\<IMPLIES\>\> | Subject prioritization |
| Student | \<\<IMPLIES\>\> | Eight hours of sleep |
| Eight hours of sleep | \<\<IMPLIES\>\> | Time allocation constraint |
| Daily schedule | \<\<ASSUMES\>\> | Consistent study environment |

---

**Research Status:** ✅ No further research is needed.

### 🧠 Draft Knowledge Graph

| **Subject** | **Predicate** | **Object** |
| :--- | :---: | :--- |
| Basic Home Hydroponic System | \<\<IMPLIES\>\> | Components |
| Basic Home Hydroponic System | \<\<IMPLIES\>\> | Acquisition Priority |
| Acquisition Priority | \<\<IMPLIES\>\> | Ordered List of Components |
| Basic Home Hydroponic System | \<\<ASSUMES\>\> | Understanding of Hydroponics |

---

**Research Status:** 🔎 **Further research is needed.**